Solving the multi-armed bandit problem using a click-through rate optimization dataset

#### What is the Multi-armed bandit problem?

First a bandit is defined as someone who steals your money. A one armed bandit is a simple slot machine wherein you insert a coin into the machine, pull a lever, and get an immediate reward.

But why is it called a bandit? Turns out most casinos configure this machines in a way that all gamblers end up lossing money.

A multi-armed bandit is a complicated slot machine wherein instead of one , there are seveeral levers which a gambler can pull, with each lever giving a different return. The probability distribution for the reward corresponding to each lever is different and is unknown to the gambler.

The task is thus to identify which lever to pull in order to get maximum reward after a given set of trials. The problem statement is like a single step Markov Decision Process. Each arm chosen is equivalent to an action, which then leads to an immediate reward.

Suppose an advertising company is running 10 different ads targeted towards a similar set of population on a webpage. We have results for which ads were clicked by a user here. Each column index represents a different ad. We have a 1 if the ad was clicked by a user, and 0 if it was not. A sample from the original dataset is shown below:

First, we will try a random selection technique, where we randomly select any ad and show it to the user. If the user clicks the ad, we get paid and if not, there is no profit.

In [3]:
# Random Selection

# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Ads_Optimisation.csv')
dataset.head()

,Ad 1,Ad 2,Ad 3,Ad 4,Ad 5,Ad 6,Ad 7,Ad 8,Ad 9,Ad 10
0,1,0,0,0,1,0,0,0,1,0
1,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0


In [4]:
dataset.shape

(10000, 10)

In [5]:
# Implementing random selection
import random
N = 10000 # Number of observcations
d = 10 # Number of ads
ads_selected = []
total_reward = 0
for n in range(0, N):
    ad = random.randrange(d)
    ads_selected.append(ad)
    reward = dataset.values[n, ad]
    total_reward = total_reward + reward

Total reward for the random selection algorithm comes out to be 1237. As this algorithm is not learning anything, it will not smartly select any ad which is giving the maximum return. And hence even if we look at the last 1000 trials, it is not able to find the optimal ad.

In [6]:
pd.Series(ads_selected).tail(1000).value_counts(normalize = True)

9    0.107
3    0.105
2    0.104
0    0.103
6    0.102
5    0.102
7    0.099
8    0.095
4    0.094
1    0.089
dtype: float64

In [7]:
# Check the total reward
total_reward

1237

$$a_t^{UCB}=argmax_{a£A}Q_t(a)+U_t(a)$$

In [9]:
# Now lets implement UCB to do the same computation.
import math
N = 10000
d = 10
ads_selected = []
num_of_selections = [0] * d
sum_of_rewards = [0] * d
total_reward = 0

for n in range(0, N):
    ad = 0
    max_upper_bound = 0
    for i in range(0, d):
        if (num_of_selections[i] > 0):
            average_reward = sum_of_rewards[i] / num_of_selections[i]
            delta_i = math.sqrt(2 * math.log(n+1) / num_of_selections[i])
            upper_bound = average_reward + delta_i
            
        else:
            upper_bound = 1e400 #select the upper bound
        if upper_bound > max_upper_bound:
            max_upper_bound = upper_bound
            ad = i # get the best ad
    ads_selected.append(ad)
    num_of_selections[ad] += 1
    reward = dataset.values[n, ad]
    sum_of_rewards[ad] += reward
    total_reward += reward # collect your total rewards

In [11]:
pd.Series(ads_selected).tail(1500).value_counts(normalize = True)

4    0.810000
0    0.077333
7    0.026667
3    0.024667
6    0.019333
2    0.019333
1    0.007333
8    0.006000
9    0.004667
5    0.004667
dtype: float64

In [12]:
# Check the total reward
total_reward

2125

The total_reward for UCB comes out to be 2125. Clearly, this is much better than random selection and indeed a smart exploration technique that can significantly improve our strategy to solve a MABP.

After just 1500 trials, UCB is already favouring Ad #5 (index 4) which happens to be the optimal ad, and gets the maximum return for the given problem.

In [16]:
len(ads_selected)

10000

In [17]:
ad

4